In [ ]:
from TransversalitySheaf import *
from ConleySheaf import *
from FlowSheaf import *
from pychomp import *
from Model import *


import matplotlib.pyplot as plt
from matplotlib import colors

In [ ]:
model_name = 'toggle'
model = Model(model_name)

In [ ]:
params = [2.52,2.52]

In [ ]:
model.simulate([.5,.5],params)

In [ ]:
base = GeometricCubicalComplex(model.base_bounds,model.base_boxes)
B = base.complex

In [ ]:
[a_min,a_max],[b_min,b_max] = base.bounds
a_vert = [a_min+i*base.delta[0] for i in range(base.boxes[0]+1)]
b_vert = [b_min+i*base.delta[1] for i in range(base.boxes[1]+1)]

In [ ]:
phase_complex = GeometricCubicalComplex(model.phase_bounds, model.phase_boxes)
X = phase_complex.complex

In [ ]:
[xmin,xmax],[ymin,ymax] = phase_complex.bounds
x_vert = [xmin+i*phase_complex.delta[0] for i in range(phase_complex.boxes[0]+1)]
y_vert = [ymin+i*phase_complex.delta[1] for i in range(phase_complex.boxes[1]+1)]

In [ ]:
C, discrete_flow = TransversalityComplex(phase_complex, model.ODE_Model,params,model.num_samples)
FC = FlowComplex(C, discrete_flow)

In [ ]:
""""
I need to define the possible values my initial points will take as they
relate to the equilibrium point. In this case I chose to plot 10 trajectories
ranging from 0.1 to 5
"""
#values = np.linspace(-1.25, 1.25, 10)
# I want each trajectory to have a different color
#vcolors = plt.cm.autumn_r(np.linspace(0.1, 1, len(values)))
 
# Open figure
f = plt.figure()
"""
I need to define a range of time over which to integrate the system of ODEs
The values don't really matter in this case because our system doesn't have t
on the right hand side of dx/dt and dy/dt, but it is a necessary input for
integrate.odeint.
"""

# Get limits of trajectory plot
#xmax,ymax = 1.25,1.25
#ymax = plt.ylim(ymin=0)[1]
#xmax = plt.xlim(xmin=0)[1]
# Define number of points
nb_points = 20
# Define x and y ranges
x = np.linspace(xmin, xmax, nb_points)
y = np.linspace(ymin, ymax, nb_points)
# Create meshgrid
X1 , Y1 = np.meshgrid(x,y)
# Calculate growth rate at each grid point
DX1, DY1 = model.ODE_Model([X1, Y1],params)
# Direction at each grid point is the hypotenuse of the prey direction and the
# predator direction.
M = (np.hypot(DX1, DY1))
# This is to avoid any divisions when normalizing
M[ M == 0] = 1.
# Normalize the length of each arrow (optional)
DX1 /= M
DY1 /= M
 
plt.title('Trajectories and direction fields')
"""
This is using the quiver function to plot the field of arrows using DX1 and
DY1 for direction and M for speed
"""
Q = plt.quiver(X1, Y1, DX1, DY1, M, pivot='mid', cmap=plt.cm.plasma)
plt.xlabel('x')
plt.ylabel('y')
#plt.legend(bbox_to_anchor=(1.05, 1.0))
plt.grid(True)
plt.xticks(x_vert)
plt.yticks(y_vert)
#plt.grid()
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.show()

In [ ]:
x_pos = []
y_pos = []
x_dir = []
y_dir = []
for u in discrete_flow.keys():
    if X.rightfringe(u): continue
    u_barys = [(bounds[1]+bounds[0])/2 for bounds in phase_complex.geometry(u)]
    #print(u,u_barys)
    for v in discrete_flow[u]:
        if X.rightfringe(v): continue
        v_barys = [(bounds[1]+bounds[0])/2 for bounds in phase_complex.geometry(v)]
        x_pos . append (u_barys[0])
        y_pos . append (u_barys[1])
        x_dir . append (v_barys[0]-u_barys[0])
        y_dir . append (v_barys[1]-u_barys[1])

In [ ]:
plt.title('Directed Graph')
"""
This is using the quiver function to plot arrows on top cells
"""
Q = plt.quiver(x_pos, y_pos, x_dir, y_dir,scale=10)
plt.xlabel('X')
plt.ylabel('Y')
plt.grid(True)
plt.xticks(x_vert)
plt.yticks(y_vert)
#plt.grid()
plt.xlim(xmin, xmax)
plt.ylim(ymin, ymax)
plt.show()

In [ ]:
GC = GradedComplexObj.induce_from_flow(FC.complex, lambda x: FC.discrete_flow[x])

In [ ]:
DrawGradedComplex(GC.graded_complex,GC.poset)

In [ ]:
#GC.poset.children(7)

In [ ]:
conley_complex = ConnectionMatrix(GC.graded_complex)
poset = GC.poset
DrawGradedComplex(conley_complex,poset)

In [ ]:
#REDUCED TOGGLE CLUSTERING

In [ ]:
model_name = 'reduced_toggle'
model = Model(model_name)

In [ ]:
base = GeometricCubicalComplex(model.base_bounds,model.base_boxes)
B = base.complex
phase_specs = [model.phase_bounds,model.phase_boxes]

In [ ]:
#%%time
#flow_sheaf = SheafFromVertices(TransversalitySheaf(base, model.ODE_Model, phase_specs, model.num_samples))

In [ ]:
%%time
conley_sheaf = ConleySheaf(GradeFlowSheaf(SheafFromVertices(TransversalitySheaf(base, model.ODE_Model, phase_specs, model.num_samples))))

In [ ]:
recurrent_sheaf = RecurrentSheaf(conley_sheaf)

In [ ]:
DrawGradedComplex(conley_sheaf.mapping[40].graded_complex,conley_sheaf.mapping[40].poset)

In [ ]:
DrawGradedComplex(recurrent_sheaf.mapping[40].graded_complex,recurrent_sheaf.mapping[40].poset)

In [ ]:
face_poset = base.face_poset

In [ ]:
face_poset

In [ ]:
def InducedPoset( G, predicate ):
    result = DirectedAcyclicGraph()
    S = set([v for v in G.vertices() if predicate(v)])
    for v in S:
        result.add_vertex(v)
    for v in S:
        for u in G.children(v):
            if u in S and u != v:
                result.add_edge(v,u)
    return Poset(result)
def subposet_iso(poset_map, A, B):
    #A,B the subposets
    img = [poset_map(v) for v in A.vertices()]
    img_set = set(img)
    if not len(img)==len(img_set):
        return False
    if not img_set==B.vertices():
        return False
    return True

In [ ]:
#Clustering algorithm, with flag for changing localization criterion
reducedFlag = True
localize = face_poset
for (u,v) in face_poset.edges():
    if B.rightfringe(u) or B.rightfringe(v): continue
    conley_map = recurrent_sheaf.mapping[(u,v)]
    gc_u = recurrent_sheaf.mapping[u]
    gc_v = recurrent_sheaf.mapping[v]
    #rc_u = InducedPoset(gc_u.poset, lambda v : v in gc_u.graded_complex.count() )
    #rc_v = InducedPoset(gc_v.poset, lambda v : v in gc_v.graded_complex.count() )
    if reducedFlag:
        if subposet_iso(conley_map.poset_map, gc_v.poset, gc_u.poset):
            #print(v,u)
            localize.add_edge(v,u)
    else:
        if conley_map.poset_iso(conley_sheaf.mapping[v],conley_sheaf.mapping[u]):
            #print(v,u)
            localize.add_edge(v,u)

In [ ]:
scc,mapping = CondensationGraph(face_poset.vertices(),lambda x : face_poset.adjacencies(x))

In [ ]:
scc

In [ ]:
graded_base = GradedComplex(base.complex, lambda x : mapping[x])

In [ ]:
DrawGradedComplex(graded_base,Poset(scc))

In [ ]:
reduced_base = ConnectionMatrix(graded_base)

In [ ]:
DrawGradedComplex(reduced_base,Poset(scc))

In [ ]:
data = np.zeros(base.boxes[0])

In [ ]:
data.shape

In [ ]:
for top_cell in B(B.dimension()):
    if B.rightfringe(top_cell): continue
    print(top_cell,base.geometry(top_cell), mapping[top_cell])
    print(recurrent_sheaf.mapping[top_cell].graded_complex.complex().size())
    print("---")
    #print(B.boundary({top_cell}))

In [ ]:
# for cell in B:
#     if B.rightfringe(cell): continue
#     print(cell,base.geometry(cell), mapping[cell])

In [ ]:
index = 69
DrawGradedComplex(recurrent_sheaf.mapping[index].graded_complex,recurrent_sheaf.mapping[index].poset)

In [ ]:
index = 70
DrawGradedComplex(recurrent_sheaf.mapping[index].graded_complex,recurrent_sheaf.mapping[index].poset)

In [ ]:
index = 19
DrawGradedComplex(recurrent_sheaf.mapping[index].graded_complex,recurrent_sheaf.mapping[index].poset)

In [ ]:
index = 20
DrawGradedComplex(recurrent_sheaf.mapping[index].graded_complex,recurrent_sheaf.mapping[index].poset)

In [ ]:
base.geometry(19)

In [ ]:
recurrent_sheaf.mapping[(69,19)].poset_map(0)

In [ ]:
index = 40
DrawGradedComplex(conley_sheaf.mapping[index].graded_complex,conley_sheaf.mapping[index].poset)

In [ ]:
#cc = conley_sheaf.mapping[12]

In [ ]:
#rc = InducedPoset(cc.poset, lambda v : v in cc.graded_complex.count() )

In [ ]:
#DrawGradedComplex(cc.graded_complex,rc)